# Data collection activities
All the data collection activities are automated using user defined functions retrievable in the folder `scripts`.

In [2]:
import time
from bs4 import BeautifulSoup
import requests as requests
from scripts import item 
from scripts import discussion 
from scripts import link
from scripts import comment
from scripts import user
from tqdm import tqdm
import sqlite3
import pandas as pd

### Fix wrongly retrieved rows
Some posts and comments have been wrongfully retieved, now they're missing the author and the amount of sats stacked.
In order to fix the error a new scraping session is needed.
The goal is:
- Collect the wrongfully scraped items
- Scrape them again with the proper setup and correct html tags

#### Comments

In [2]:
# Spot the wrongfully retrieved rows for the Comments table
conn = sqlite3.connect('../data/stacker_news.sqlite')
wrong_comments = """
SELECT *
FROM comments
WHERE Author=='None' AND Sats LIKE '@%';
"""

retrieve_wrong_comments = pd.read_sql(wrong_comments, conn)

retrieve_wrong_comments = pd.DataFrame(retrieve_wrong_comments)

conn.close()

In [3]:
retrieve_wrong_comments

,ItemCode,Sats,Boost,Comments,Author,Tag,Timestamp,CommentsItemCode
0,28,@satoshisuncle 12 Jun 2021,1 boost,1 reply,None,freebie,2021-06-12 17:26:20,[29]
1,37,@satoshisuncle OP 14 Jun 2021,None,0 replies,None,freebie,2021-06-14 18:30:33,[]
2,64,@satoshisuncle 17 Jun 2021,None,0 replies,None,freebie,2021-06-17 22:07:33,[]
3,1312,@satoshi_std OP 23 Aug 2021,None,0 replies,None,freebie,2021-08-23 14:30:19,[]
4,1708,@satoshisuncle OP 3 Sep 2021,None,0 replies,None,freebie,2021-09-03 22:49:38,[]
...,...,...,...,...,...,...,...,...
1695,269426,@designsats 28 Sep,None,1 reply,None,meta,2023-09-28 21:26:33,[269427]
1696,269543,@satoshi_in_the_classroom OP 29 Sep,None,0 replies,None,bitcoin,2023-09-29 04:46:06,[]
1697,269544,@satoshi_in_the_classroom OP 29 Sep,None,0 replies,None,bitcoin,2023-09-29 04:46:19,[]
1698,269546,@satoshi_in_the_classroom OP 29 Sep,None,2 replies,None,bitcoin,2023-09-29 04:46:43,"[269550, 269552]"


#### Posts

In [4]:
# Spot the wrongfully retrieved rows for the Post table
conn = sqlite3.connect('../data/stacker_news.sqlite')

wrong_posts = """
SELECT *
FROM post
WHERE Author=='None' AND Sats LIKE '@%';
"""

retrieve_wrong_posts = pd.read_sql(wrong_posts, conn)

retrieve_wrong_posts = pd.DataFrame(retrieve_wrong_posts)

conn.close()

In [5]:
retrieve_wrong_posts

,Title,Category,ItemCode,Sats,Boost,Comments,Author,Tag,Timestamp,MainLink,BodyLinks,SatsReceivedComments,CommentsItemCode
0,Fedcoin is coming,link,31,@satoshisuncle 13 Jun 2021,None,2 comments,None,bitcoin,2021-06-13 15:36:08,None,['https://www.politico.com/news/2021/06/12/fed...,1 sat,"[32, 46]"
1,Why Altcoins aren't copying Taproot. Bitcoin T...,link,34,@satoshisuncle 14 Jun 2021,None,0 comments,None,bitcoin,2021-06-14 18:17:21,None,['https://jimmysong.substack.com/p/why-altcoin...,None,[]
2,Bitrefill's Work in El Salvador,link,35,@satoshisuncle 14 Jun 2021,None,0 comments,None,bitcoin,2021-06-14 18:23:46,None,['https://twitter.com/bitrefill/status/1402624...,None,[]
3,3 Ways El Salvador Has Changed Bitcoin Forever,link,36,@satoshisuncle 14 Jun 2021,None,1 comment,None,bitcoin,2021-06-14 18:27:33,None,['https://medium.com/tantra-labs/3-ways-el-sal...,0 sats,[37]
4,Lightning Builder's Guide: Understanding Liqui...,link,92,@satoshisuncle 25 Jun 2021,None,1 comment,None,bitcoin,2021-06-25 18:01:11,None,['https://docs.lightning.engineering/the-light...,1 sat,[98]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,Tether Bitcoin Mining Moria Instrument Demonst...,link,268075,@satsyoshi 27 Sep,None,1 comment,None,bitcoin,2023-09-27 14:13:14,None,['https://u.today/tether-bitcoin-mining-moria-...,0 sats,[268103]
1127,Hopes Of A Spot Bitcoin ETF Dashed As SEC Agai...,link,268337,@satsyoshi 27 Sep,None,3 comments,None,bitcoin,2023-09-27 15:53:04,None,['https://zycrypto.com/hopes-of-a-spot-bitcoin...,0 sats,"[268415, 268399, 268346]"
1128,A reminder to purposely USE Bitcoin and Lightn...,discussion,268523,@satoshi_in_the_classroom 27 Sep,None,8 comments,None,bitcoin,2023-09-27 18:39:05,None,[],35 sats,"[268529, 268627, 268628, 268549, 268629, 26863..."
1129,What Happens When Bitcoin Miners Mine an Inval...,link,268678,@satsyoshi 27 Sep,None,1 comment,None,bitcoin,2023-09-27 23:07:00,None,['https://beincrypto.com/marathon-digital-mine...,0 sats,[268680]


#### General list of items that must be scraped again

In [6]:
retrieve = list(retrieve_wrong_posts['ItemCode']) + list(retrieve_wrong_comments['ItemCode'])

In [7]:
# Queries for entry insertion in tables
insert_comment = """
INSERT OR IGNORE INTO comments (
    ItemCode,
    Sats,
    Boost,
    Comments,
    Author,
    Tag,
    Timestamp,
    CommentsItemCode
    ) values (?, ?, ?, ?, ?, ?, ?, ?)
"""

insert_post = """
INSERT OR IGNORE INTO post (
    Title,
    Category,
    ItemCode,
    Sats,
    Boost,
    Comments,
    Author,
    Tag,
    Timestamp,
    MainLink,
    BodyLinks,
    SatsReceivedComments,
    CommentsItemCode
    ) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

insert_exception = """
INSERT OR IGNORE INTO exceptions (
    RequestResult,
    ItemCode,
    Soup
    ) values (?, ?, ?)
"""

### Setup the fixing functions

In [8]:
conn = sqlite3.connect('../data/stacker_news.sqlite')
cur = conn.cursor()

for i in tqdm(retrieve):
    try:
        # Provided a string returns a bs4.BeautifulSoup object
        url_posts = f'https://stacker.news/items/{i}'
        response = requests.get(url_posts)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        if item.detect_item_type(i, soup) == 'comment':
            # Insert every new entry into a new row in the provided DB
            entry = (str(i),
                     str(comment.extract_banner(soup)['sats']),
                     str(comment.extract_banner(soup)['boost']),
                     str(comment.extract_banner(soup)['comments']),
                     str(comment.extract_banner(soup)['author']),
                     str(comment.extract_banner(soup)['tag']),
                     str(comment.extract_banner(soup)['timestamp']),
                     str(comment.extract_comment_item_code(soup))
                     )
            try:
                cur.execute(insert_comment, entry)
            except:
                print(f'Error while inserting the comment item {i} in the database')

        elif item.detect_item_type(i, soup) == 'link':
            # Appends every new profile to a csv file in the provided path
            entry = (str(link.extract_title(soup)),
                     str(item.detect_item_type(i, soup)),
                     str(i),
                     str(link.extract_banner(soup)['sats']),
                     str(link.extract_banner(soup)['boost']),
                     str(link.extract_banner(soup)['comments']),
                     str(link.extract_banner(soup)['author']),
                     str(link.extract_banner(soup)['tag']),
                     str(link.extract_banner(soup)['timestamp']),
                     str(link.extract_link(soup)),
                     str(link.extract_body_links(soup)),
                     str(link.extract_comment_stacked(soup)),
                     str(link.extract_comment_item_code(soup))
                     )
            try:
                cur.execute(insert_post, entry)
            except:
                print(f'Error while inserting the link item {i} in the database')

        elif item.detect_item_type(i, soup) in ['discussion', 'poll', 'bounty']:
            entry = (str(discussion.extract_title(soup)),
                     str(item.detect_item_type(i, soup)),
                     str(i),
                     str(discussion.extract_banner(soup)['sats']),
                     str(discussion.extract_banner(soup)['boost']),
                     str(discussion.extract_banner(soup)['comments']),
                     str(discussion.extract_banner(soup)['author']),
                     str(discussion.extract_banner(soup)['tag']),
                     str(discussion.extract_banner(soup)['timestamp']),
                     None,
                     str(discussion.extract_body_links(soup)),
                     str(discussion.extract_comment_stacked(soup)),
                     str(discussion.extract_comment_item_code(soup))
                     )

            # Appends every new profile to a csv file in the provided path
            try:
                cur.execute(insert_post, entry)
            except:
                print(f'Error while inserting the post item {i} in the database')

        if i % 1000 == 0:
            conn.commit()
            time.sleep(0.5)
            continue
    except:

        try:
            exception_entry = (
                str(response),
                str(i),
                str(soup)
            )

            cur.execute(insert_exception, exception_entry)

            # If the request is not authorized than stop the scraping because I've been probably blocked
            if response.status_code == 403 or response.status_code == 401:
                exit()
        except:
            continue

# Final commit
conn.commit()

# Close connection to DB
cur.close()
conn.close()

100%|██████████| 2831/2831 [31:23<00:00,  1.50it/s]


## Profile scraping

**NB**: this code must be run after the end of the whole scraping activity because an `unique(author)` is needed in order to scrape all the user profiles in the forum. 

**The `unique(author)` must be the result of a `UNION ALL` between the tables.**

In [3]:
conn = sqlite3.connect('../data/stacker_news.sqlite')

query = """
SELECT DISTINCT Author
FROM (
    SELECT Author
    FROM comments
    UNION ALL
    SELECT Author
    FROM post
     );
"""

sql_query = pd.read_sql(query, conn)
result = pd.DataFrame(sql_query,
                      columns=['Author'])

conn.close()


In [5]:
conn = sqlite3.connect('../data/stacker_news.sqlite')
cur = conn.cursor()

sql_user = """
DROP TABLE IF EXISTS user;
CREATE TABLE user (
    User TEXT,
    TotalStacked TEXT,
    FirstItem TEXT,
    HatStreak TEXT,
    NumItems TEXT,
    PRIMARY KEY (User))
"""

cur.executescript(sql_user)

conn.commit()
conn.close()

In [6]:
insert_user = """
INSERT INTO user (
    User,
    TotalStacked,
    FirstItem,
    HatStreak,
    NumItems
    ) values (?, ?, ?, ?, ?)
"""

In [7]:
conn = sqlite3.connect('../data/stacker_news.sqlite')
cur = conn.cursor()

for i in tqdm(result['Author']):
    try:
        profile_data = user.get_profile(i)
        entry = (
            str(profile_data[0]),
            str(profile_data[1]),
            str(profile_data[2]),
            str(profile_data[3]),
            str(profile_data[4])
        )
        try:
            cur.execute(insert_user, entry)
        except:
            continue
    except:
        continue

conn.commit()
cur.close()
conn.close()

100%|██████████| 5883/5883 [1:03:41<00:00,  1.54it/s]
